In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np

In [2]:
# Read results data set
results = pd.read_csv('results.csv')
results.head()

,date,year,home_team,away_team,home_score,away_score,total_goals,win_margin,tournament,city,country,neutral
0,2000-01-04,2000,Egypt,Togo,2,1,3,1,Friendly,Aswan,Egypt,False
1,2000-01-07,2000,Tunisia,Togo,7,0,7,7,Friendly,Tunis,Tunisia,False
2,2000-01-08,2000,Trinidad and Tobago,Canada,0,0,0,0,Friendly,Port of Spain,Trinidad and Tobago,False
3,2000-01-09,2000,Burkina Faso,Gabon,1,1,2,0,Friendly,Ouagadougou,Burkina Faso,False
4,2000-01-09,2000,Guatemala,Armenia,1,1,2,0,Friendly,Los Angeles,United States,True


#### 1. Count the Number of Unique Home Teams and Away Teams

In [3]:
unique_home_teams = results.groupby('home_team').agg(
                    {'home_team' : 'nunique'})
unique_home_teams_count = unique_home_teams.count()
unique_home_teams_count

home_team    245
dtype: int64

In [4]:
unique_away_teams = results.groupby('away_team').agg(
                    {'away_team':'nunique'})
unique_away_teams_count = unique_away_teams.count()
unique_away_teams_count

away_team    242
dtype: int64

#### 2. Tournament Statistics

For each tournament
- Compute the number of games played
- Sum up the total number of goals 
- Obtain the maximum and minimum win margins

In [5]:
games_played = results.groupby('tournament').agg(
                                             {'date':'count',
                                              'total_goals':'sum',
                                              'win_margin': ['max','min']
                                             })
games_played.columns = ['number_of_games_played','total_number_of_goals','max_win_margin','min_win_margin']
games_played= games_played.reset_index()
games_played

,tournament,number_of_games_played,total_number_of_goals,max_win_margin,min_win_margin
0,AFC Asian Cup qualification,465,1480,20,-8
1,African Cup of Nations,421,933,5,-5
2,African Cup of Nations qualification,1012,2388,7,-10
3,CECAFA Cup,341,858,9,-9
4,FIFA World Cup,384,965,8,-6
5,FIFA World Cup qualification,4942,14174,31,-12
6,Friendly,7472,18853,15,-9
7,Gold Cup,297,796,7,-6
8,UEFA Euro qualification,1293,3593,11,-13
9,UEFA Nations League,468,1139,6,-6


#### 3. Analyzing the 2022 FIFA World Cup

Calculate the Total Number of Goals by Each Team in the FIFA World Cup 2022. 

Hint: We'll need to combine each team's goals scored as both the home team and away team. 

1. **Split** `results` into a DataFrame containing only FIFA World Cup 2022 matches
- Combine multiple boolean masks using the `&` operator:
    - `results['tournament']=='FIFA World Cup'`
    - `results['year']== 2022` 

- Perform a second split that creates separate DataFrames for home teams and away teams

2. **Apply** aggregation functions to each DataFrame that sums up the total number of goals scored by each team
- Don't forget to flatten the index of the resulting aggregated DataFrames

3. **Combine** the two DataFrames by performing a **left** join where the DataFrame on the left contains the data for the home team (important!)
- Feel free to clean the column names **but don't drop the `home_team` and `away_team` columns!**
- Create a new column `total_goals` that adds up the total goals scored by each team as the home team and away team
- Sort from highest to lowest number of `total_goals` by each team.

In [6]:
fifa_world = results[((results['tournament'] == 'FIFA World Cup') & (results['year']==2022))]
fifa_total_home_score = fifa_world.groupby('home_team').agg({'home_score':'sum'})
fifa_total_home_score.columns = ['total_home_score']
fifa_total_home_score = fifa_total_home_score.reset_index()

fifa_total_away_score = fifa_world.groupby('away_team').agg({'away_score':'sum'})
fifa_total_away_score.columns = ['total_away_score']
fifa_total_away_score = fifa_total_away_score.reset_index()

total_scores = pd.merge(left=fifa_total_home_score,
                        right=fifa_total_away_score,
                        left_on='home_team',
                        right_on='away_team',
                        how='left')
total_scores['total_goals'] = total_scores['total_home_score']+total_scores['total_away_score']
total_scores.sort_values(by='total_goals',ascending=False)


,home_team,total_home_score,away_team,total_away_score,total_goals
11,France,11,France,5.0,16.0
0,Argentina,11,Argentina,4.0,15.0
10,England,10,England,3.0,13.0
20,Portugal,11,Portugal,1.0,12.0
18,Netherlands,6,Netherlands,4.0,10.0
26,Spain,8,Spain,1.0,9.0
3,Brazil,7,Brazil,1.0,8.0
7,Croatia,7,Croatia,1.0,8.0
17,Morocco,1,Morocco,5.0,6.0
12,Germany,1,Germany,5.0,6.0


Note: Since the world cup in 2022 took place in Qatar, they never played as the away team! This is why we used the **left** join in order to return Qatar's total home goals!

Feel free to continue exploring the dataset by adding more cells below!